## MLflow Implementation
MLflow is a comprehensive platform that can logs experiments with detailed metadata by recording parameters, metrics, model version and output artifacts, it allows to seamlessly manage the machine learning lifecycle making comparative analysis with a clear UI.

In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import mlflow
import dagshub
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### First experiment

In [ ]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [ ]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

**In this case we need to restore the experiment**

In [ ]:
# Create the MLflow client
client = mlflow.tracking.MlflowClient()

# Get the experiment by its name
experiment = client.get_experiment_by_name("First Experiment")

if experiment is None:
    print("Experiment not found.")
else:
    # Restore the experiment using its ID
    client.restore_experiment(experiment.experiment_id)
    print(f"Experiment '{experiment.name}' has been restored.")

Experiment 'First Experiment' has been restored.


In [ ]:
mlflow.set_experiment("First Experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr, "Logistic Regression") 

2025/02/18 10:18:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run clumsy-shark-19 at: http://127.0.0.1:5000/#/experiments/247970322603174652/runs/1a073c5d913c482aa8a97ef645d11388
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/247970322603174652


### Experiment 1: Train Logistic Regression Classifier

In [5]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [6]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



### Experiment 3: Train XGBoost

In [7]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [8]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [9]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



### Track Experiments Using MLFlow

In [10]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [11]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [12]:
report

{'0': {'precision': 0.9814126394052045,
  'recall': 0.9777777777777777,
  'f1-score': 0.9795918367346939,
  'support': 270.0},
 '1': {'precision': 0.8064516129032258,
  'recall': 0.8333333333333334,
  'f1-score': 0.819672131147541,
  'support': 30.0},
 'accuracy': 0.9633333333333334,
 'macro avg': {'precision': 0.8939321261542151,
  'recall': 0.9055555555555556,
  'f1-score': 0.8996319839411174,
  'support': 300.0},
 'weighted avg': {'precision': 0.9639165367550067,
  'recall': 0.9633333333333334,
  'f1-score': 0.9635998661759786,
  'support': 300.0}}

In [ ]:
# Create the MLflow client
client = mlflow.tracking.MlflowClient()

# Retrieve the experiment by name
experiment = client.get_experiment_by_name("Anomaly Detection")

if experiment is None:
    print("Experiment not found.")
else:
    # Restore the experiment
    client.restore_experiment(experiment.experiment_id)
    print(f"Experiment '{experiment.name}' has been restored.")

Experiment 'Anomaly Detection' has been restored.


In [ ]:
mlflow.set_experiment("Anomaly Detection")
# Look at the repository in MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/02/18 10:22:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/235694836365368649/runs/3ad4aadc67d7472da65a974825bf2656
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/235694836365368649


2025/02/18 10:22:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/235694836365368649/runs/d5dca81e20c44aad8d9f748876df4efe
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/235694836365368649


2025/02/18 10:23:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/235694836365368649/runs/f2f4bf000b3f4e789060d6fa79753ddc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/235694836365368649


2025/02/18 10:23:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/235694836365368649/runs/69abac4306d7495aadd1e6d0c0849901
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/235694836365368649


### **Register the Model**

Registering a model in MLflow is a crucial step in the machine learning lifecycle because it enables effective model versioning, management, and deployment. Here are the key reasons why registering a model is important:

---

**1. Centralized Model Management**
- **Version Control:** Each registered model is automatically assigned a unique version (e.g., "Model v1," "Model v2"). This allows you to track and manage changes as models are retrained or improved.
- **Organization:** Centralizes all models in a registry, making it easier to manage multiple models across different projects.

---

**2. Model Deployment and Serving**
- Registered models can be directly deployed to various environments (e.g., REST API endpoints, cloud services).
- MLflow supports **model serving**, where a registered model can be served for real-time predictions using built-in or custom MLflow environments.

---

**3. Reproducibility**
- Registering a model saves the exact version of the model along with its associated artifacts (e.g., training code, environment, parameters). This ensures that the model can be consistently reproduced across environments and users.

---

**4. Governance and Lifecycle Management**
- MLflow supports managing the **lifecycle stages** of a model:
  - `Staging`: A model is in the testing phase before being promoted to production.
  - `Production`: The model is actively serving in production.
  - `Archived`: The model is no longer actively used but retained for reference.
- This stage management enables proper governance and risk control during deployment.

---

**5. Collaboration**
- Teams can share and collaborate on registered models across projects and departments. This is especially useful for large organizations where multiple teams might reuse models.
- Different team members can promote models to different stages (e.g., Data Scientists register and test the model, while DevOps promotes it to production).

---

**6. Experiment Comparison**
- MLflow allows you to compare multiple model versions and registered models to evaluate performance, parameters, and metrics across experiments. This helps with selecting the best model for deployment.

---

**7. Integration with CI/CD Pipelines**
- Registered models can be integrated into continuous integration/continuous deployment (CI/CD) pipelines. This helps automate the deployment process, ensuring smooth transitions from development to production.

---

**Conclusion**
Registering a model in MLflow is a best practice for model lifecycle management, improving collaboration, governance, reproducibility, and deployment readiness. It plays a key role in operationalizing machine learning (MLOps) and ensuring models are effectively tracked and deployed across different environments.

In [ ]:
model_name = "XGB-Classifier"
run_id = input("Enter run ID:")
model_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri, model_name
)

Registered model 'XGB-Classifier' already exists. Creating a new version of this model...
2025/02/18 12:07:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Classifier, version 2
Created version '2' of model 'XGB-Classifier'.


### Load the Model

In [ ]:
model_name = "XGB-Classifier"
model_version = 2
model_uri = f"models/{model_name}@challenger"

mlflow.set_tracking_uri("http://127.0.0.1:5000")
loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:5]

OSError: No such file or directory: 'models\XGB-Classifier\challenger'

In [31]:
model_uri = "mlflow-artifacts:/235694836365368649/f2f4bf000b3f4e789060d6fa79753ddc/artifacts/model"

loaded_model = mlflow.xgboost.load_model(model_uri)

# Predict using the loaded model
y_pred = loaded_model.predict(X_test)
print(y_pred[:5])

[0 0 0 0 0]


### Transition the Model to Production


In [37]:
current_model_uri = "models:/XGB-Classifier/2"
production_model_name = "Anomaly-Detection-Prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Registered model 'Anomaly-Detection-Prod' already exists. Creating a new version of this model...
Copied version '2' of model 'XGB-Classifier' to version '4' of model 'Anomaly-Detection-Prod'.


<ModelVersion: aliases=[], creation_timestamp=1739892351145, current_stage='None', description='', last_updated_timestamp=1739892351145, name='Anomaly-Detection-Prod', run_id='f2f4bf000b3f4e789060d6fa79753ddc', run_link='', source='models:/XGB-Classifier/2', status='READY', status_message=None, tags={}, user_id='', version='4'>

In [38]:
client.get_registered_model(production_model_name).latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1739892351145, current_stage='None', description='', last_updated_timestamp=1739892351145, name='Anomaly-Detection-Prod', run_id='f2f4bf000b3f4e789060d6fa79753ddc', run_link='', source='models:/XGB-Classifier/2', status='READY', status_message=None, tags={}, user_id='', version='4'>,
 <ModelVersion: aliases=['production'], creation_timestamp=1739891345759, current_stage='Production', description='', last_updated_timestamp=1739891435986, name='Anomaly-Detection-Prod', run_id='f2f4bf000b3f4e789060d6fa79753ddc', run_link='', source='models:/XGB-Classifier/1', status='READY', status_message=None, tags={}, user_id='', version='3'>]

In [39]:
# Transition the model to production or champion stage
client.transition_model_version_stage(
    name=production_model_name,    # Model name
    version=4,                      # Version of the model to promote
    stage='Production'                # The stage to promote to
)

<ModelVersion: aliases=[], creation_timestamp=1739892351145, current_stage='Production', description='', last_updated_timestamp=1739892481040, name='Anomaly-Detection-Prod', run_id='f2f4bf000b3f4e789060d6fa79753ddc', run_link='', source='models:/XGB-Classifier/2', status='READY', status_message=None, tags={}, user_id='', version='4'>

In [41]:
# Download Production Model
# model_version = 1
# prod_model_uri = f"models:/{production_model_name}@production"

mlflow.set_tracking_uri("http://127.0.0.1:5000")
loaded_model = mlflow.xgboost.load_model("models:/Anomaly-Detection-Prod/Production")
y_pred = loaded_model.predict(X_test)
y_pred[:5]

array([0, 0, 0, 0, 0])

### DagsHub

Combining DagsHub with MLflow creates a powerful, end-to-end ecosystem for managing your data science projects. Each tool addresses complementary aspects of the workflow, and together they provide a unified framework for version control, experiment tracking, collaboration, and reproducibility. Here’s a detailed look at the advantages:

---

**1. Unified Project Management**

- **Version Control for Code and Data:**  
  - **DagsHub:** Utilizes Git for code versioning and integrates with DVC (Data Version Control) to manage datasets and large files.  
  - **Benefit:** You maintain a clear history of code changes and data modifications, ensuring that every experiment is associated with a specific state of both code and data.
  
- **Experiment Tracking with MLflow:**  
  - **MLflow:** Focuses on tracking experiment parameters, metrics, and artifacts (such as models, visualizations, or logs).  
  - **Benefit:** You get a detailed log of each experiment run, making it easier to compare and reproduce results.

- **Integrated Workflow:**  
  - **Combination Advantage:** By linking MLflow’s experiment runs to specific Git commits and DVC-tracked data snapshots on DagsHub, you achieve a high level of traceability. This means that any experiment can be traced back to the exact code and dataset version used, which is invaluable for debugging and auditing.

---

**2. Enhanced Collaboration and Transparency**

- **Team Collaboration:**  
  - **DagsHub:** Provides collaborative features such as pull requests, issue tracking, and shared repositories, facilitating teamwork and code reviews.  
  - **MLflow:** When used in tandem, the experiment tracking data becomes accessible to the team, allowing everyone to review model performance, compare experiments, and discuss improvements.

- **Transparency in Model Development:**  
  - **Benefit:** The combined setup offers a transparent view of how models evolve over time. Stakeholders can see which changes in the code or data led to improvements (or degradations) in performance, supporting better decision-making.


**3. Seamless Transition from Research to Production**

- **Model Registry and Deployment:**  
  - **MLflow Model Registry:** Helps in managing different versions of your models, including those developed for tasks like anomaly detection.
  - **DagsHub Integration:** With all changes tracked and versioned, transitioning a well-performing model from research to production becomes much smoother, as you can reliably reference the exact code, data, and experiment details associated with that model.

- **Continuous Integration/Continuous Deployment (CI/CD):**  
  - **Benefit:** Integrating these tools into your CI/CD pipelines ensures that improvements are automatically tested and deployed, further enhancing operational efficiency.

---

**Conclusion**

By combining DagsHub and MLflow, you create an environment where every aspect of your data science project—from code and data management to experiment tracking and collaboration—is seamlessly integrated. This approach not only enhances reproducibility and auditability but also fosters efficient teamwork and continuous improvement, ultimately leading to more robust and reliable models in real-life projects. Whether you're developing anomaly detection systems or any other type of model, this integrated setup can significantly streamline your workflow and elevate your project management practices.

In [46]:
# Set up DagsHub
dagshub.init(repo_owner='SebastianGarrido2790', repo_name='mlflow_dagshub_demo', mlflow=True)

Initialized MLflow to track repo "SebastianGarrido2790/mlflow_dagshub_demo"

Repository SebastianGarrido2790/mlflow_dagshub_demo initialized!

In [48]:
# Ideally you will not require following 4 lines if you have started fresh and do not have any previous dagshub credentials on your computer
# import os
# os.environ['MLFLOW_TRACKING_USERNAME'] = "SebastianGarrido2790"
# os.environ['MLFLOW_TRACKING_PASSWORD'] = "30391249ad213d2d5844a26146b6319d7b09d7ed"
# os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/SebastianGarrido2790/mlflow_dagshub_demo.mlflow" 

# Publish to our centralize Dgashub server
mlflow.set_experiment("Anomaly Detection")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/02/18 13:14:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/235694836365368649/runs/36455769418245d6a5d999c56d69d03d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/235694836365368649


2025/02/18 13:15:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/235694836365368649/runs/e34cf5c0f3e74cc1ade1161ce2d42f49
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/235694836365368649


2025/02/18 13:15:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/235694836365368649/runs/575c988ea5024086a0fbf510f60d4e04
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/235694836365368649


2025/02/18 13:16:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/235694836365368649/runs/a99db691e2244753bfe16207572839e4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/235694836365368649
